# 端到端建模流程
## 从原始数据输入到结果文件生成

# 0.引入三方库

In [26]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error
from scipy import sparse
from sklearn.model_selection import KFold

# 1.读取所需数据

In [27]:
debug = False
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv', nrows = NROWS)[['card_id', 'target']]
test =  pd.read_csv('data/test.csv', nrows = NROWS)[['card_id']]
cols = ['card_id', 'merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'city_id']
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)[cols].astype(str)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)[cols].astype(str)

transaction = pd.concat([new_transaction, history_transaction], axis=0, ignore_index=True)
# 减少内存起见，统一对merchant和transaction的id列进行编码
merchant_id = transaction['merchant_id'].unique().tolist()
merchant_map = pd.Series(range(len(merchant_id)), index=merchant_id)
transaction['merchant_id'] = transaction['merchant_id'].map(merchant_map).astype(str)
new_transaction['merchant_id'] = new_transaction['merchant_id'].map(merchant_map).astype(str)
history_transaction['merchant_id'] = history_transaction['merchant_id'].map(merchant_map).astype(str)

# 2.做数据预处理

In [28]:

for co in ['merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'city_id']:
    print(co)

    temp = new_transaction.groupby("card_id")[co].apply(list).apply(lambda x:' '.join(x)).reset_index()
    temp.columns = ['card_id', co+'_new']
    train = pd.merge(train, temp, how='left', on='card_id')
    test = pd.merge(test, temp, how='left', on='card_id')

    temp = history_transaction.groupby("card_id")[co].apply(list).apply(lambda x:' '.join(x)).reset_index()
    temp.columns = ['card_id', co+'_hist']
    train = pd.merge(train, temp, how='left', on='card_id')
    test = pd.merge(test, temp, how='left', on='card_id')

    temp = transaction.groupby("card_id")[co].apply(list).apply(lambda x:' '.join(x)).reset_index()
    temp.columns = ['card_id', co+'_all']
    train = pd.merge(train, temp, how='left', on='card_id').fillna("-1")
    test = pd.merge(test, temp, how='left', on='card_id').fillna("-1")

merchant_id
merchant_category_id
state_id
subsector_id
city_id


# 3.进行特征提取

In [34]:
train_x = pd.DataFrame()
test_x = pd.DataFrame()

cntv = CountVectorizer()

tfv = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
    
    
vector_feature =[]
for co in ['merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'city_id']:
    vector_feature.extend([co+'_new', co+'_hist', co+'_all'])
for feature in vector_feature:
    print(feature)
    cntv.fit(train[feature].append(test[feature]))
    train_x = sparse.hstack((train_x, cntv.transform(train[feature]))).tocsr()
    test_x = sparse.hstack((test_x, cntv.transform(test[feature]))).tocsr()
    
    tfv.fit(train[feature].append(test[feature]))
    train_x = sparse.hstack((train_x, cntv.transform(train[feature]))).tocsr()
    test_x = sparse.hstack((test_x, cntv.transform(test[feature]))).tocsr()

merchant_id_new
merchant_id_hist
merchant_id_all
merchant_category_id_new
merchant_category_id_hist
merchant_category_id_all
state_id_new
state_id_hist
state_id_all
subsector_id_new
subsector_id_hist
subsector_id_all
city_id_new
city_id_hist
city_id_all


In [35]:
sparse.save_npz('preprocess/train_x_tfv_cntv.npz', train_x)
sparse.save_npz('preprocess/test_x_tfv_cntv.npz', test_x)

# 4.进行模型训练及预测

In [36]:
train_y = pd.read_csv('data/train.csv', nrows = NROWS)['target']

In [37]:
params_initial = {
    'num_leaves':31, 
    'learning_rate':0.01, 
'boosting':'gbdt',
'min_child_samples':20,
'bagging_seed': 2020,
'bagging_fraction':0.7, 
'bagging_freq':1,
'feature_fraction':0.7, 
 'max_depth':-1,
    'metric':'rmse',
'reg_alpha':0,
'reg_lambda':1, 
'objective':'regression'
    }
print(train_x.shape)
print(test_x.shape)
ESR = 30
NBR = 10000
VBE = 50


testarr = 0
trainse = pd.Series()
skf = KFold(n_splits=5, random_state=2020, shuffle=True)
score = []
for train_part_index, evals_index in skf.split(train_x, train_y):
    EVAL_RESULT = {}
    train_part = lgb.Dataset(train_x[train_part_index, :], label=train_y.loc[train_part_index])
    evals = lgb.Dataset(train_x[evals_index, :], label=train_y.loc[evals_index])
    bst = lgb.train(params_initial, train_part,
                    num_boost_round=NBR, valid_sets=[train_part, evals],
                    valid_names=['train', 'evals'], early_stopping_rounds=ESR,
                    evals_result=EVAL_RESULT, verbose_eval=VBE)
    testarr += bst.predict(test_x)
    trainse = trainse.append(pd.Series(bst.predict(train_x[evals_index, :]), index=evals_index))
    score.append(bst.best_score['evals']['rmse'])
print(score, sum(score)/5)

pd.Series(trainse.sort_index().values).to_csv("preprocess/train_tfv_cntv.csv", index=False)
pd.Series(testarr/5).to_csv("preprocess/test_tfv_cntv.csv", index=False)

(201917, 1778094)
(123623, 1778094)
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.82433	evals's rmse: 3.84201
[100]	train's rmse: 3.80485	evals's rmse: 3.8361
[150]	train's rmse: 3.78804	evals's rmse: 3.83295
[200]	train's rmse: 3.7731	evals's rmse: 3.82999
[250]	train's rmse: 3.76066	evals's rmse: 3.82873
[300]	train's rmse: 3.74977	evals's rmse: 3.82839
[350]	train's rmse: 3.74011	evals's rmse: 3.8282
Early stopping, best iteration is:
[340]	train's rmse: 3.74197	evals's rmse: 3.82817
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.83128	evals's rmse: 3.81816
[100]	train's rmse: 3.8123	evals's rmse: 3.81145
[150]	train's rmse: 3.79613	evals's rmse: 3.80782
[200]	train's rmse: 3.782	evals's rmse: 3.80489
[250]	train's rmse: 3.76956	evals's rmse: 3.80318
[300]	train's rmse: 3.75883	evals's rmse: 3.8026
[350]	train's rmse: 3.74943	evals's rmse: 3.80228
Early stopping, best iteration is:
[350]	train's rmse: 3.74943	eva

(201917, 1774092)
(123623, 1774092)
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.83029	evals's rmse: 3.84615
[100]	train's rmse: 3.81656	evals's rmse: 3.8432
[150]	train's rmse: 3.80551	evals's rmse: 3.842
[200]	train's rmse: 3.79572	evals's rmse: 3.84158
Early stopping, best iteration is:
[201]	train's rmse: 3.79554	evals's rmse: 3.84154
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.83734	evals's rmse: 3.8216
[100]	train's rmse: 3.82396	evals's rmse: 3.81749
[150]	train's rmse: 3.81297	evals's rmse: 3.81542
[200]	train's rmse: 3.80362	evals's rmse: 3.81473
Early stopping, best iteration is:
[188]	train's rmse: 3.80577	evals's rmse: 3.81465
Training until validation scores don't improve for 30 rounds
[50]	train's rmse: 3.82732	evals's rmse: 3.86276
[100]	train's rmse: 3.81418	evals's rmse: 3.85833
[150]	train's rmse: 3.80352	evals's rmse: 3.85602
[200]	train's rmse: 3.79467	evals's rmse: 3.85522
[250]	train's rmse

In [ ]:
# 增加所有字段的tfv和cntv

In [24]:
debug = True
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv', nrows = NROWS)[['card_id', 'target']]
test =  pd.read_csv('data/test.csv', nrows = NROWS)[['card_id']]
cols = ['card_id', 'merchant_id']
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)

In [25]:
new_transaction.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id'],
      dtype='object')

In [40]:
debug = False
NROWS = 10000 if debug else None
train = pd.read_csv('data/train.csv', nrows = NROWS)[['card_id', 'target']]
test =  pd.read_csv('data/test.csv', nrows = NROWS)[['card_id']]
cols = ['card_id', 'merchant_id', 'merchant_category_id', 'state_id', 'subsector_id', 'city_id']
new_transaction = pd.read_csv('data/new_merchant_transactions.csv', nrows = NROWS)
history_transaction = pd.read_csv('data/historical_transactions.csv', nrows = NROWS)

In [41]:
new_transaction.groupby('card_id')['purchase_date'].min().apply(lambda x:time.)

card_id
C_ID_00007093c1    2018-04-03 11:13:35
C_ID_0001238066    2018-03-01 16:48:27
C_ID_0001506ef0    2018-03-16 22:21:58
C_ID_0001793786    2017-11-15 15:44:20
C_ID_000183fdda    2018-03-02 12:26:26
C_ID_0002709b5a    2018-03-02 11:00:50
C_ID_000298032a    2018-03-29 16:16:37
C_ID_0002ba3c2e    2017-11-01 18:33:30
C_ID_0002c7c2c1    2018-02-07 17:43:32
C_ID_00032df08f    2018-01-05 15:13:57
C_ID_0003754056    2018-02-11 15:15:48
C_ID_000377f6a0    2018-03-30 15:54:22
C_ID_0003be3c83    2018-03-10 14:50:28
C_ID_0003f41435    2018-03-11 09:49:10
C_ID_00042d509c    2018-02-08 16:24:50
C_ID_0004587331    2018-02-08 18:48:43
C_ID_0004725b87    2018-04-26 15:34:20
C_ID_0004888ddd    2018-03-09 09:11:54
C_ID_0004b68c49    2018-01-10 09:17:49
C_ID_0004c2a5ab    2018-03-02 18:25:15
C_ID_00057b99fe    2018-03-29 15:05:30
C_ID_000599daf9    2018-03-06 08:33:35
C_ID_0005f16cc8    2018-03-10 13:11:17
C_ID_0006152db8    2018-03-07 01:53:10
C_ID_000616f4a8    2018-03-19 14:18:52
C_ID_000664aa02  